## About
This is a Python notebook reproducing the example in Olfsson et al.'s *Good practices for estimating area and assessing accuracy of land change* (2014) for calculating the following statistics in a land classification map:
- producer's accuracies and user's accuracies by class,
- overall accuracy, and
- area estimates. 

Each statistic is accompanied by a 95% confidence interval. 

## References

Pontus Olofsson, Giles M. Foody, Martin Herold, Stephen V. Stehman, Curtis E. Woodcock, Michael A. Wulder, Good practices for estimating area and assessing accuracy of land change, Remote Sensing of Environment,Volume 148, 2014, Pages 42-57, ISSN 0034-4257, https://doi.org/10.1016/j.rse.2014.02.015.  (https://www.sciencedirect.com/science/article/pii/S0034425714000704)

Perrot, M., & Duchesnay, É. (2011). Scikit-learn: Machine Learning in Python. Journal of
Machine Learning Research, 12, 2825--2830.

In [1]:
import os
import pandas as pd
import numpy as np

## Data


Here we extract data from table 8 in Olfosson et al. (2014) to form the  confusion matrix $n$ such that

$n_{i,j}$ = number of points predicted as class $i$, known to be class $j$, 

which is equivalent to

$n_{i,j}$ = number of points that have map class as $i$ and reference class $j$.

In [5]:
# Olofsson et al. Table 8 data 

# number of classes in the map
n_classes = 4

# ---------------------------
# pixels per class in the map
pix_counts = np.array([200000, 150000, 3200000, 6450000])

print("Pixel counts per class:")
for n_pix, i  in zip(pix_counts, range(n_classes)):
    print(f'class {i+1}: {pix_counts[i]:>12,}')
print('\n')

# ---------------------------
# confusion matrix
n = np.array([
    [66, 0,  5,   4],
    [0,  55, 8,   12],
    [1,  0,  153, 11],
    [2,  1,  9,   313]])
    
print('Confusion matrix:\n',n)

Pixel counts per class:
class 1:      200,000
class 2:      150,000
class 3:    3,200,000
class 4:    6,450,000
[[ 66   0   5   4]
 [  0  55   8  12]
 [  1   0 153  11]
 [  2   1   9 313]]


### Notation
Throughout the following, let 
- $q$ be the number of map classes, 
- $p_{ij}$ be the (true) fraction of the map that has map class $i$ and reference class $j$,
- $p_{i\cdot} = \sum_{j=1}^q p_{ij}$ be the fraction of the map classified as class $i$, and
- $p_{\cdot j} = \sum_{i=1}^q p_{ij}$ be the fraction of the class that has reference class $j$. 

### User's Accuracy
The user's accuracy of class $i$ is the fraction of the area mapped as class $i$ that has reference class $i$.
This can be calculated as (Olofsson et al., eq. 2):
$$U_i = \frac{p_{ii}}{p_{i\cdot}}.$$
This is equivalent to the *precision* of class $i$. For example, when there are two classes (positive and negative) the user's accuracy of the positive class is the same as the precision of the true class, TP/(TP + FP).

Let $\hat{U}_i$ be the estimate of $U_i$'s from the our sampled points. We have that
$$\hat{U}_i = \frac{\hat{p}_{ii}}{\hat{p}_{i\cdot}},$$
where $\hat{p}_{ij}$ are the estimations of $p_{ij}$ from the sample. 
For stratified random sampling in which the sampling strata correspond to the map classes we have that
$$\hat{p}_{ij} = W_i \frac{n_{ij}}{n_{i\cdot}},$$
where
- $W_i$ is the fraction of the map's area classified  as class $i$,
- $n_{ij}$ is number of points with map class $i$, known to be reference class $j$ (entries in the confusion matrix), and
- $n_{i\cdot}$ is the number of points with map class $i$ (row sums in confusion matrix).

Notice that the user's accuracy can be simplifeid to
$$\hat{U}_i = \frac{n_{ii}}{n_{i\cdot}}.$$
This is the formula implemented in the code.

For the user's accuracy of map class $i$, the estimated variance is (Olfosson et al., eq. 6):
$$\hat{V}(\hat{U}_i) = \frac{\hat{U}_i (1-\hat{U}_i)}{n_{i\cdot}-1}.$$

In this notbeokoj we calculate the user's accuracies first (before the overall accuracy) since these are needed to calculate the approximate variance of the overall accuracy. 

### Standard Error & Confidence Intervals
The square root of the estimated variance results in the standard error of the estimator (Olofsson et al., sec. 4.3). 
For example, in the case of an estimated user's accuracy $\hat{U}_i$ we have that $\hat{S}(\hat{U}_i) = \sqrt{\hat{V}(\hat{U}_i)}$.

The standard error is then used to get confidence intervals for the estimated statistic. The 95% confidence interval is estimated as (Olofsson et al., sec. 5.2.1)
$$\hat{U}_i \pm 1.96 \hat{S}(\hat{U}_i) = \hat{U}_i \pm 1.96\  \sqrt{\hat{V}(\hat{U}_i)}.$$
We can replace $\hat{U}_i$ for the overall accuracy or producer's accuracy estimates to obtain the corresponding variances, standard errors, and confidence intervals. 

In the follwoing cells we will calculate $\hat{U}_i$ with a 95% confidence interval.

In [6]:
# points in sample that had class i in map (predicted as i, any true class j)
n_idot = [sum(n[i,:]) for i in range(n_classes)]

# -------------------------------------
# estimated user's accuracy
U_hat = [n[i,i] / n_idot[i] for i in range(n_classes)]

# variance of estimated user's accuracy
var_U_hat = [U_hat[i] * (1-U_hat[i])/(n_idot[i]-1) for i in range(n_classes)]

# -------------------------------------
print("User's accuracies with 95% confidence interval:")
for U_hati, var_i, i  in zip(U_hat, var_U_hat, range(n_classes)):
    print(f'class {i+1}: {U_hati:.2f} ± {1.96 * np.sqrt(var_i):.2f}')

User's accuracies with 95% confidence interval
class 1: 0.88 ± 0.07
class 2: 0.73 ± 0.10
class 3: 0.93 ± 0.04
class 4: 0.96 ± 0.02


### Overall Accuracy

Let $O$ be the (true) accuracy of the map, and $\hat{O}$ its estimation from the sample. Then, following Olofsson et al., section 4.3, we have that 
$$\hat{O} = \sum_{i=1}^q \hat{p}_{ii} = \sum_{i=1}^q W_i \frac{n_{ii}}{n_{i\cdot}}.$$
Recall that $q$ is the number of classes in the map and $\hat{p}_{ii}$ is the estimation of $p_{ii}$, the (true) fraction of the area in the map that was classified as class $i$ and has reference class $i$. 

For overall accuracy, the estimated variance is (Olofsson et al., eq. 5):
$$\hat{V}(\hat{O}) = \sum_{i=1}^q \frac{W_i^2 \hat{U}_i (1-\hat{U}_i)}{n_{i\cdot}-1}.$$

In the follwoing cells we will calculate $\hat{O}$ with a 95% confidence interval.

In [7]:
# total number of pixels in the map
total_pix = sum(pix_counts)

# fractions of area in map classified in each class
W = [pix_counts[i]/ total_pix for i in range(n_classes)]      

# -------------------------------------
# estimated overall accuracy
O_hat = sum([W[i]*n[i,i]/n_idot[i] for i in range(n_classes)])

# -------------------------------------
# variance of estimated overall accuracy
var_O_hat = sum([ W[i]**2 * U_hat[i] * (1-U_hat[i])/(n_idot[i]-1) for i in range(n_classes)])

# -------------------------------------
print('Overall accuracy with 95% confidence interval:')
print(f'{O_hat:.2f} ± {1.96 * np.sqrt(var_O_hat):.2f}')

Overall accuracy with 95% confidence interval
0.95 ± 0.02


### Producer's Accuracy
The producer's accuracy of class $i$ is the fraction of the (true) area with reference class $i$ that is actually mapped as class $j$. 
We can calculate it as (Olofsson et al., eq. 3):
$$P_j = \frac{p_{jj}}{p_{\cdot j}}.$$
This is equivalent to the sensitiviy of class $j$. For example, when there are two classes (positive and negative) the producer's accuracy of the positive class is the same as the sensitivy of the true class (TP/(TP + FN)).

The estimate of $P_j$ from the sampled points is
$$\hat{P}_j = \frac{\hat{p}_{jj}}{\hat{p}_{\cdot j}},$$
where the $\hat{p}_{ij}$ are as before.

The estimated variance of the estimated producer's accuracy of class $j$ is given by (Olofsson et al., eq. 7):

$$\hat{V}(\hat{P}_j) = 
\frac{1}{\hat{N}_{\cdot j}^2} 
\left( 
\frac{N_{j \cdot}^2 (1 - \hat{P}_j)^2 \hat{U}_j (1-\hat{U}_j)}{n_{j \cdot} -1}  
+
\hat{P}_j^2
\sum_{i\neq j}^q 
N_{i\cdot}^2 
\frac{n_{ij}}{(n_{i \cdot} - 1)n_{i \cdot}} 
\left( 1 - \frac{n_{ij}}{n_{i \cdot}} \right)
\right),$$
where
- $N_{j \cdot}$ is the number of pixels in the map with map class $j$,
- $n_{j\cdot}$ is the number of sample points with map class $j$, and
- $\hat{N}_{\cdot j} = \sum_{i=1}^q \frac{N_{i\cdot}}{n_{i\cdot}}n_{ij}$ is the estimated number of pixels with reference class $j$.

In the following cells we calculate $\hat{P}_j$ with a 95% confidence interval.

In [13]:
# estimated fraction of area in class j (Olofsson et al., eq. 9) 
p_hat_dotj = []
for j in range(n_classes):
    p_hat_ij = [ W[i]*n[i,j]/n_idot[i] for i in range(n_classes) ]
    p_hat_dotj.append(sum(p_hat_ij))  
    
# -------------------------------------
# estimated producer's accuracy 
P_hat = [ (W[j]*n[j,j]/n_idot[j]) / p_hat_dotj[j] for j in range(n_classes)]

# -------------------------------------
# variance of estimated producer's accuracy 
# notice N_jdot is pix_counts[j]
N_hat_dotj = []
for j in range(n_classes):
    summands = [ pix_counts[i] * n[i,j]/n_idot[i] for i in range(n_classes)]
    N_hat_dotj.append(sum(summands))

summand1 = []
for j in range(n_classes):
    summand1.append((pix_counts[j]**2) * ((1-P_hat[j])**2) * U_hat[j] * (1-U_hat[j]) / (n_idot[j] - 1))

summand2 = []
for j in range(n_classes):
    inner = []
    for i in range(n_classes):
        if i!=j:
            inner.append( (pix_counts[i]**2) * (n[i,j])/(n_idot[i]) * ( 1 - n[i,j]/n_idot[i])/(n_idot[i]-1)) 
    summand2.append((P_hat[j]**2) * sum(inner))

var_P_hat = [1/(N_hat_dotj[j]**2) *  (summand1[j] + summand2[j]) for j in range(n_classes)]

# -------------------------------------
print("Producer's accuracies with 95% confidence interval:")
for P_hati, var_i, i  in zip(P_hat, var_P_hat, range(n_classes)):
    print(f'class {i+1}: {P_hati:.2f} ± {1.96 * np.sqrt(var_i):.2f}')

Producer's accuracies with 95% confidence interval:
class 1: 0.75 ± 0.21
class 2: 0.85 ± 0.25
class 3: 0.93 ± 0.03
class 4: 0.96 ± 0.02


### Area Estimates

For stratified random sampling when the map classes are the strata, an estimator of the fraction of area of class $j$ is (Olofsson et al., eq. 9):
$$ \hat{p}_{\cdot j} = \sum_{i=1}^q W_i \frac{n_{ij}}{n_{i\cdot}}.$$

For this estimator of area fraction per class, the standard error is estimated by (Olofsson et al. eq 10):
$$S(\hat{p}_{\cdot j}) =  
\sqrt{
\sum_{i=1}^q W_i^2 \frac{ \frac{n_{ij}}{n_{i\cdot}} \left(1 -  \frac{n_{ij}}{n_{i\cdot}} \right)}{n_{i \cdot}-1}
}.$$

The estimated area of class $j$ is thus
$$\hat{A}_j = A \times \hat{p}_{\cdot k},$$
where $A$ is the total are of the map. 
The standard error for the area is given by (Olofsson et al. eq 11):
$$ S(\hat{A}_j) = A \times S(\hat{p}_{\cdot j}).$$

In the following celss we estimate the area covered by each class together with 95% confidence intervals. 
Recall we had already estimated the fraction of area per class $\hat{p_{\cdot j}}$ for the producer's accuracy.


In [10]:
# standard error of estimated area per class
S_p_hat_dotj = []
for j in range(n_classes):
    summands = [ (W[i]**2) * (n[i,j]/n_idot[i]) * (1 -  (n[i,j]/n_idot[i]))/ (n_idot[i]-1) 
                for i in range(n_classes)]
    S_p_hat_dotj.append(np.sqrt(sum(summands)))
    
# -------------------------------------
print("Percentage area per class with 95% confidence interval:")
for perc_i, err_i, i  in zip(p_hat_dotj, S_p_hat_dotj, range(n_classes)):
    print(f'class {i+1}: {perc_i:.3f} ± {1.96 * err_i:.4f}')

Percentage area per class with 95% confidence interval
class 1: 0.024 ± 0.0068
class 2: 0.013 ± 0.0042
class 3: 0.318 ± 0.0172
class 4: 0.646 ± 0.0181


In [12]:
# area per pixel
pixel_area_m2 = 30**2

# total map area in hectares (1 m^2 = 1/10^4 ha)
map_area_ha = total_pix * pixel_area_m2 / 10**4

approx_area_per_class = [map_area_ha * p_hat_dotj[i] for i in range(n_classes)]

# standard error area per class
SE_area_per_class = [map_area_ha * S_p_hat_dotj[i] for i in range(n_classes)]

# -------------------------------------
print("Estimated area per class (ha) with 95% confidence interval:")
for area_i, err_i, i  in zip(approx_area_per_class, SE_area_per_class, range(n_classes)):
    print(f'class {i+1}: {area_i:,.0f} ± {1.96 * err_i:,.0f}')

Estimated area per class (ha) with 95% confidence interval
class 1: 21,158 ± 6,158
class 2: 11,686 ± 3,756
class 3: 285,770 ± 15,510
class 4: 581,386 ± 16,282
